In [ ]:
Create and Art with Neural style transfer on given image using deep learning. 

In [2]:
!pip install tensorflow matplotlib pillow
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Function to load and preprocess images
def load_and_process_image(path, max_dim=512):
    img = Image.open(path)
    long = max(img.size)
    scale = max_dim / long
    img = img.resize((round(img.size[0] * scale), round(img.size[1] * scale)))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.expand_dims(img, axis=0)
    return tf.keras.applications.vgg19.preprocess_input(img)

# Function to deprocess image back to normal display
def deprocess(img):
    x = img.copy()
    x = x.reshape((x.shape[1], x.shape[2], 3))
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# Load content and style images
content_path = 'content.jpg'  # replace with your content image path
style_path = 'style.jpg'      # replace with your style image path

content_image = load_and_process_image(content_path)
style_image = load_and_process_image(style_path)

# Load VGG19 model
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
vgg.trainable = False

# Define layers to use
content_layers = ['block5_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

# Model to extract style and content
def get_model():
    outputs = [vgg.get_layer(name).output for name in style_layers + content_layers]
    model = tf.keras.Model([vgg.input], outputs)
    return model

# Compute Gram matrix for style representation
def gram_matrix(input_tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1] * input_shape[2], tf.float32)
    return result / num_locations

# Extract features
def get_features(model, content, style):
    content_outputs = model(content)
    style_outputs = model(style)

    style_features = [gram_matrix(style_layer) for style_layer in style_outputs[:num_style_layers]]
    content_features = [content_layer for content_layer in content_outputs[num_style_layers:]]
    return style_features, content_features

# Compute loss
def compute_loss(model, loss_weights, init_image, gram_style_features, content_features):
    style_weight, content_weight = loss_weights
    model_outputs = model(init_image)

    style_output_features = model_outputs[:num_style_layers]
    content_output_features = model_outputs[num_style_layers:]

    style_score = 0
    content_score = 0

    for target, comb in zip(gram_style_features, style_output_features):
        style_score += tf.reduce_mean((gram_matrix(comb) - target) ** 2)

    for target, comb in zip(content_features, content_output_features):
        content_score += tf.reduce_mean((comb - target) ** 2)

    style_score *= style_weight / num_style_layers
    content_score *= content_weight / num_content_layers
    total_loss = style_score + content_score
    return total_loss

# Run style transfer
def run_style_transfer(content_path, style_path, num_iterations=1000, content_weight=1e3, style_weight=1e-2):
    model = get_model()
    for layer in model.layers:
        layer.trainable = False

    content_image = load_and_process_image(content_path)
    style_image = load_and_process_image(style_path)

    gram_style_features, content_features = get_features(model, content_image, style_image)

    init_image = tf.Variable(content_image, dtype=tf.float32)
    opt = tf.optimizers.Adam(learning_rate=5)

    best_loss, best_img = float('inf'), None

    for i in range(num_iterations):
        with tf.GradientTape() as tape:
            loss = compute_loss(model, (style_weight, content_weight), init_image, gram_style_features, content_features)

        grads = tape.gradient(loss, init_image)
        opt.apply_gradients([(grads, init_image)])
        init_image.assign(tf.clip_by_value(init_image, -103.939, 255.0 - 103.939))

        if loss < best_loss:
            best_loss = loss
            best_img = init_image.numpy()

        if i % 100 == 0:
            print(f"Iteration {i}, Loss: {loss.numpy()}")

    return deprocess(best_img)

# Run and display
output = run_style_transfer(content_path, style_path, num_iterations=500)
plt.imshow(output)
plt.axis('off')
plt.show()


You should consider upgrading via the 'C:\Users\ayush\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ayush\\Desktop\\CL3\\content.jpg'